In [0]:
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
lr_ctr = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/lr_ctr.csv')

In [0]:
rf_ctr = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/fixed_features/rf_ctr_130.csv')

In [0]:
xg_ctr = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/xg_ctr_90.csv')

In [0]:
xg_ctr = xg_ctr.drop(columns = ['Unnamed: 0'])

In [0]:
gbdt_ctr = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/fixed_features/gbdt_ctr_130.csv')

In [0]:
validation = pd.read_csv('/content/gdrive/My Drive/we_data/FINAL_CSVs/FINAL_VALIDATION_v1.csv')

In [0]:
validation.head()

,Unnamed: 0,click,hour,bidid,userid,city,slotprice,bidprice,payprice,domain1,...,slotsize_620*60,slotsize_640*90,slotsize_728*90,slotsize_760*90,slotsize_910*90,slotsize_950*90,slotsize_960*60,slotsize_960*90,slotsize_980*80,slotsize_980*90
0,0,0,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,79,0,300,23,-2.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,79,0,238,75,0.0,...,0,0,0,0,0,1,0,0,0,0
2,2,0,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,2,5,238,65,0.0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,205,5,238,6,-1.0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,135,5,249,5,-2.0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
def linear(ctr, validation):
  avg_ctr = ctr.mean()  #注意这里的avg 不是 202/impressions
  base_range = np.arange(60, 82, 1)

  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  
  for base in base_range:
    current_bid = base * ctr/avg_ctr
    budget = 6250 * 1000
    clicks = 0
    impressions = 0
   
    for j in range(len(current_bid)):
      payprice = validation.payprice[j]
      if current_bid[j] > payprice:
        budget = budget - payprice
        if budget >= 0:
          impressions += 1
          if(validation.click[j] == 1):
            clicks += 1
        else:
          budget = budget + payprice
          break
          
    click_list.append(clicks)
    budget_list.append(budget)
    impression_list.append(impressions)
    bid_price.append(current_bid)
    print('base = {}, clicks = {}, impression = {}, budget = {}'.format(base,clicks, impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
  base_optimal = base_range[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
  print('Maxium is {}, when base = {}, budget = {}'.format(max_click, base_optimal, budget_max, impression_max))
  
  return max_click, base_optimal, bid_price_final

# Linear LR 

In [0]:
max_click_lr, c_optimal_lr, bid_price_lr = linear(lr_ctr['click_proba'], validation)

base = 72.5, clicks = 133,, impression = 136430, impression = 136430, budget = 13
Maxium is 133, when base = 72.5, budget = 13


In [0]:
'''
base = 60, clicks = 136, budget = 1307102
base = 61, clicks = 136, budget = 1243725
base = 62, clicks = 139, budget = 966815
base = 63, clicks = 140, budget = 916197
base = 64, clicks = 140, budget = 842270
base = 65, clicks = 140, budget = 783772
base = 66, clicks = 143, budget = 253692
base = 67, clicks = 144, budget = 218039
base = 68, clicks = 141, budget = 51
base = 69, clicks = 138, budget = 4
base = 70, clicks = 136, budget = 112
base = 71, clicks = 133, budget = 125
base = 72, clicks = 133, budget = 2
base = 73, clicks = 132, budget = 34
base = 74, clicks = 132, budget = 13
base = 75, clicks = 127, budget = 52
base = 76, clicks = 126, budget = 17
base = 77, clicks = 124, budget = 72
base = 78, clicks = 122, budget = 15
'''

# Linear XG

In [0]:
max_click, c_optimal, bid_price_xg = linear(xg_ctr['click_proba'], validation)

base = 60, clicks = 158, impression = 105130, budget = 1814443
base = 61, clicks = 159, impression = 107168, budget = 1693898
base = 62, clicks = 161, impression = 109201, budget = 1573310
base = 63, clicks = 162, impression = 111343, budget = 1441554
base = 64, clicks = 162, impression = 113347, budget = 1320180
base = 65, clicks = 163, impression = 115467, budget = 1184475
base = 66, clicks = 163, impression = 117510, budget = 1054018
base = 67, clicks = 163, impression = 119487, budget = 924593
base = 68, clicks = 163, impression = 121451, budget = 794468
base = 69, clicks = 165, impression = 123553, budget = 657015
base = 70, clicks = 165, impression = 125728, budget = 511806
base = 71, clicks = 165, impression = 127920, budget = 358655
base = 72, clicks = 165, impression = 130040, budget = 210939
base = 73, clicks = 165, impression = 132091, budget = 71406
base = 74, clicks = 164, impression = 132463, budget = 48
base = 75, clicks = 159, impression = 131569, budget = 40
base = 76,

In [0]:
'''
XG
base = 40, clicks = 132, budget = 4334911
base = 50, clicks = 141, budget = 3136289
base = 60, clicks = 155, budget = 1805068
base = 70, clicks = 164, budget = 431425
base = 80, clicks = 141, budget = 35
base = 90, clicks = 124, budget = 6
base = 100, clicks = 102, budget = 43
base = 110, clicks = 91, budget = 38
base = 120, clicks = 80, budget = 33
base = 130, clicks = 74, budget = 14

base = 60, clicks = 155, budget = 1805068
base = 61, clicks = 157, budget = 1675516
base = 62, clicks = 157, budget = 1550500
base = 63, clicks = 157, budget = 1397327
base = 64, clicks = 160, budget = 1263270
base = 65, clicks = 161, budget = 1112724
base = 66, clicks = 162, budget = 983919
base = 67, clicks = 162, budget = 861363
base = 68, clicks = 162, budget = 722941
base = 69, clicks = 164, budget = 585974
base = 70, clicks = 164, budget = 431425
base = 71, clicks = 164, budget = 273031
base = 72, clicks = 165, budget = 112403
base = 73, clicks = 163, budget = 244
base = 74, clicks = 158, budget = 53
base = 75, clicks = 149, budget = 32
base = 76, clicks = 146, budget = 2
base = 77, clicks = 145, budget = 69
base = 78, clicks = 143, budget = 13
base = 79, clicks = 142, budget = 6
base = 80, clicks = 141, budget = 35
base = 81, clicks = 138, budget = 0
Maxium is 165, when base = 72, budget = 0

130 features
base = 60, clicks = 158, impression = 105130, budget = 1814443
base = 61, clicks = 159, impression = 107168, budget = 1693898
base = 62, clicks = 161, impression = 109201, budget = 1573310
base = 63, clicks = 162, impression = 111343, budget = 1441554
base = 64, clicks = 162, impression = 113347, budget = 1320180
base = 65, clicks = 163, impression = 115467, budget = 1184475
base = 66, clicks = 163, impression = 117510, budget = 1054018
base = 67, clicks = 163, impression = 119487, budget = 924593
base = 68, clicks = 163, impression = 121451, budget = 794468
base = 69, clicks = 165, impression = 123553, budget = 657015
base = 70, clicks = 165, impression = 125728, budget = 511806
base = 71, clicks = 165, impression = 127920, budget = 358655
base = 72, clicks = 165, impression = 130040, budget = 210939
base = 73, clicks = 165, impression = 132091, budget = 71406
base = 74, clicks = 164, impression = 132463, budget = 48
base = 75, clicks = 159, impression = 131569, budget = 40
base = 76, clicks = 154, impression = 130383, budget = 11
base = 77, clicks = 150, impression = 129427, budget = 96
base = 78, clicks = 149, impression = 128432, budget = 49
base = 79, clicks = 147, impression = 127513, budget = 37
base = 80, clicks = 147, impression = 126555, budget = 12
base = 81, clicks = 145, impression = 125712, budget = 26
Maxium is 165, when base = 69, budget = 657015
'''

# Linear RF

In [0]:
max_click_rf, c_optimal_rf,bid_price_final_rf = linear(rf_ctr['click_proba'], validation)

base = 60, clicks = 152, impression = 109490, budget = 1799714
base = 61, clicks = 153, impression = 111636, budget = 1671031
base = 62, clicks = 153, impression = 113789, budget = 1539521
base = 63, clicks = 154, impression = 116002, budget = 1403984
base = 64, clicks = 156, impression = 118148, budget = 1267990
base = 65, clicks = 156, impression = 120197, budget = 1135350
base = 66, clicks = 156, impression = 122260, budget = 1001504
base = 67, clicks = 157, impression = 124379, budget = 862761
base = 68, clicks = 158, impression = 126556, budget = 722177
base = 69, clicks = 159, impression = 128626, budget = 589667
base = 70, clicks = 161, impression = 130679, budget = 452512
base = 71, clicks = 163, impression = 132854, budget = 307056
base = 72, clicks = 164, impression = 134951, budget = 164339
base = 73, clicks = 164, impression = 136982, budget = 25018
base = 74, clicks = 160, impression = 136630, budget = 19
base = 75, clicks = 155, impression = 135675, budget = 17
base = 76,

ValueError: ignored

In [0]:
'''
base = 60, clicks = 146, budget = 1711720
base = 61, clicks = 146, budget = 1591649
base = 62, clicks = 147, budget = 1479969
base = 63, clicks = 148, budget = 1368063
base = 64, clicks = 148, budget = 1251448
base = 65, clicks = 148, budget = 1133272
base = 66, clicks = 150, budget = 1025847
base = 67, clicks = 152, budget = 912441
base = 68, clicks = 155, budget = 793891
base = 69, clicks = 155, budget = 682363
base = 70, clicks = 156, budget = 559824
base = 71, clicks = 159, budget = 436884
base = 72, clicks = 159, budget = 316675
base = 73, clicks = 160, budget = 196210
base = 74, clicks = 161, budget = 80134
base = 75, clicks = 160, budget = 50
base = 76, clicks = 155, budget = 14
base = 77, clicks = 151, budget = 10

base = 74, clicks = 161, budget = 80134

130
base = 60, clicks = 152, impression = 109490, budget = 1799714
base = 61, clicks = 153, impression = 111636, budget = 1671031
base = 62, clicks = 153, impression = 113789, budget = 1539521
base = 63, clicks = 154, impression = 116002, budget = 1403984
base = 64, clicks = 156, impression = 118148, budget = 1267990
base = 65, clicks = 156, impression = 120197, budget = 1135350
base = 66, clicks = 156, impression = 122260, budget = 1001504
base = 67, clicks = 157, impression = 124379, budget = 862761
base = 68, clicks = 158, impression = 126556, budget = 722177
base = 69, clicks = 159, impression = 128626, budget = 589667
base = 70, clicks = 161, impression = 130679, budget = 452512
base = 71, clicks = 163, impression = 132854, budget = 307056
base = 72, clicks = 164, impression = 134951, budget = 164339
base = 73, clicks = 164, impression = 136982, budget = 25018
base = 74, clicks = 160, impression = 136630, budget = 19
base = 75, clicks = 155, impression = 135675, budget = 17
base = 76, clicks = 150, impression = 134655, budget = 4
base = 77, clicks = 147, impression = 133693, budget = 15
base = 78, clicks = 146, impression = 132689, budget = 7
base = 79, clicks = 143, impression = 131722, budget = 15
base = 80, clicks = 142, impression = 130862, budget = 14
base = 81, clicks = 143, impression = 129965, budget = 3
Maxium is 164, when base = 72, budget = 164339
'''

# Linear GBDT

In [0]:
max_click_gbdt, c_optimal_gbdt, bid_price_final_gbdt = linear(gbdt_ctr['click_proba'], validation)

base = 60, clicks = 156, impression = 99654, budget = 1552708
base = 61, clicks = 157, impression = 101673, budget = 1423237
base = 62, clicks = 157, impression = 103708, budget = 1290377
base = 63, clicks = 160, impression = 105780, budget = 1156153
base = 64, clicks = 162, impression = 107779, budget = 1015920
base = 65, clicks = 163, impression = 109726, budget = 881891
base = 66, clicks = 166, impression = 111704, budget = 742866
base = 67, clicks = 167, impression = 113755, budget = 598141
base = 68, clicks = 167, impression = 115696, budget = 462775
base = 69, clicks = 167, impression = 117641, budget = 327411
base = 70, clicks = 168, impression = 119631, budget = 189540
base = 71, clicks = 168, impression = 121636, budget = 46998
base = 72, clicks = 164, impression = 121816, budget = 133
base = 73, clicks = 160, impression = 121179, budget = 26
base = 74, clicks = 153, impression = 120452, budget = 40
base = 75, clicks = 151, impression = 119833, budget = 7
base = 76, clicks = 1

In [0]:
'''
selected features
base = 60, clicks = 148, budget = 1930005
base = 61, clicks = 149, budget = 1795775
base = 62, clicks = 150, budget = 1675989
base = 63, clicks = 153, budget = 1559516
base = 64, clicks = 153, budget = 1361148
base = 65, clicks = 153, budget = 1198609
base = 66, clicks = 154, budget = 1058019
base = 67, clicks = 154, budget = 887453
base = 68, clicks = 154, budget = 720090
base = 69, clicks = 154, budget = 596649
base = 70, clicks = 154, budget = 434547
base = 71, clicks = 155, budget = 265979
base = 72, clicks = 157, budget = 135626
base = 73, clicks = 157, budget = 65
base = 74, clicks = 153, budget = 26
base = 75, clicks = 146, budget = 20
base = 76, clicks = 142, budget = 39
base = 77, clicks = 140, budget = 6
base = 78, clicks = 139, budget = 43
base = 79, clicks = 138, budget = 123
base = 80, clicks = 137, budget = 24
base = 81, clicks = 135, budget = 40
Maxium is 157, when base = 72, budget = 135626

130features
base = 60, clicks = 156, impression = 99654, budget = 1552708
base = 61, clicks = 157, impression = 101673, budget = 1423237
base = 62, clicks = 157, impression = 103708, budget = 1290377
base = 63, clicks = 160, impression = 105780, budget = 1156153
base = 64, clicks = 162, impression = 107779, budget = 1015920
base = 65, clicks = 163, impression = 109726, budget = 881891
base = 66, clicks = 166, impression = 111704, budget = 742866
base = 67, clicks = 167, impression = 113755, budget = 598141
base = 68, clicks = 167, impression = 115696, budget = 462775
base = 69, clicks = 167, impression = 117641, budget = 327411
base = 70, clicks = 168, impression = 119631, budget = 189540
base = 71, clicks = 168, impression = 121636, budget = 46998
base = 72, clicks = 164, impression = 121816, budget = 133
base = 73, clicks = 160, impression = 121179, budget = 26
base = 74, clicks = 153, impression = 120452, budget = 40
base = 75, clicks = 151, impression = 119833, budget = 7
base = 76, clicks = 149, impression = 119112, budget = 56
base = 77, clicks = 147, impression = 118351, budget = 39
base = 78, clicks = 147, impression = 117655, budget = 9
base = 79, clicks = 145, impression = 116973, budget = 35
base = 80, clicks = 144, impression = 116391, budget = 28
base = 81, clicks = 140, impression = 115750, budget = 120
Maxium is 168, when base = 70, budget = 189540
'''

In [0]:

# def non_linear(ctr, validation):
#   range_all = np.arange(0, 100, 10)
#   lamda = 0.001
#   click_list = []
#   budget_list = []
#   impression_list = []
#   bid_price = []
#   for c in range_all:
#     current_bid = c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
#                      -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3)))    
# #     current_bid =  ((c/lamda) * ctr + c**2) **0.5 - c   #non_linear 2
# #     print(current_bid)
#     budget = 6250 * 1000
#     clicks = 0
#     impressions = 0
    
#     for j in range(len(current_bid)):
#       payprice = validation.payprice[j]
#       if current_bid[j] > payprice:
#         budget = budget - payprice
#         if budget >= 0:
#           impressions += 1
#           if(validation.click[j] == 1):
#             clicks += 1
#         else:
#           budget = budget + payprice
#     click_list.append(clicks)
#     budget_list.append(budget)
#     impression_list.append(impressions)
#     bid_price.append(current_bid)
# #     print("\n")
#     print('C = {}, clicks = {}, impression = {}, budget = {}'.format(c, clicks, impressions, budget))
    
#   max_click = max(click_list)
#   index_click = click_list.index(max_click)
#   c_optimal = range_all[index_click]
#   budget_max = budget_list[index_click]
#   impression_max = impression_list[index_click]
#   bid_price_final = bid_price[index_click]
#   print('Maxium is {}, when c = {}, budget = {}, impression = {}'.format(max_click, c_optimal, budget_max, impression_max))

#   return max_click, c_optimal, bid_price_final

In [0]:
# for xgboost
def non_linear(ctr, validation):
  range_all = np.arange(1, 21, 1)
  lamda = 0.001
  click_list = []
  budget_list = []
  impression_list = []
  bid_price = []
  for c in range_all:
    current_bid = c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3)))    
#     current_bid =  ((c/lamda) * ctr + c**2) **0.5 - c   #non_linear 2
#     print(current_bid)
    budget = 6250 * 1000
    clicks = 0
    impressions = 0
    
    for j in range(len(current_bid)):
      payprice = validation.payprice[j]
      if current_bid[j] > payprice:
        budget = budget - payprice
        if budget >= 0:
          impressions += 1
          if(validation.click[j] == 1):
            clicks += 1
        else:
          budget = budget + payprice
    click_list.append(clicks)
    budget_list.append(budget)
    impression_list.append(impressions)
    bid_price.append(current_bid)
#     print("\n")
    print('C = {}, clicks = {}, impression = {}, budget = {}'.format(c, clicks, impressions, budget))
    
  max_click = max(click_list)
  index_click = click_list.index(max_click)
  c_optimal = range_all[index_click]
  budget_max = budget_list[index_click]
  impression_max = impression_list[index_click]
  bid_price_final = bid_price[index_click]
  print('Maxium is {}, when c = {}, budget = {}, impression = {}'.format(max_click, c_optimal, budget_max, impression_max))

  return max_click, c_optimal, bid_price_final

# Non-linear XG

In [0]:
max_click_n_xg, c_optimal_n_xg, bid_price_final = non_linear(xg_ctr['click_proba'], validation)

C = 0, clicks = 0, impression = 0, budget = 6250000
C = 10, clicks = 45, impression = 66388, budget = 5093394
C = 20, clicks = 80, impression = 90568, budget = 4101808
C = 30, clicks = 97, impression = 119023, budget = 2407663
C = 40, clicks = 113, impression = 138353, budget = 1109717
C = 50, clicks = 121, impression = 150368, budget = 228929
C = 60, clicks = 115, impression = 148771, budget = 0
C = 70, clicks = 122, impression = 143319, budget = 0
C = 80, clicks = 117, impression = 139471, budget = 0
C = 90, clicks = 117, impression = 136873, budget = 0
Maxium is 122, when c = 70, budget = 0, impression = 143319


In [0]:
'''
XGboost

current_bid = ((c/lamda) * ctr + c**2) **0.5 -c  
Maxium is 150, when c = 510, budget = 2092738, lamda = 0.001

lamda = 0.0001 maxium is 107 


    current_bid = c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3))) 
  lamda = 0.001                   
C = 300, clicks = 145,, impression = 84022, budget = 2782283
C = 310, clicks = 145,, impression = 83523, budget = 2809307
C = 320, clicks = 146,, impression = 83154, budget = 2825678
C = 330, clicks = 144,, impression = 82802, budget = 2841861
C = 340, clicks = 143,, impression = 82470, budget = 2856517
C = 350, clicks = 142,, impression = 82009, budget = 2881781
C = 360, clicks = 142,, impression = 81682, budget = 2898108
C = 370, clicks = 142,, impression = 81241, budget = 2921308
C = 380, clicks = 142,, impression = 80959, budget = 2935383

    current_bid = 1.2*c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3)))  
                     
C = 100, clicks = 155, impression = 116878, budget = 1373473
C = 110, clicks = 158, impression = 116970, budget = 1296168
C = 120, clicks = 158, impression = 116901, budget = 1235676
C = 130, clicks = 160, impression = 116621, budget = 1194801
C = 140, clicks = 160, impression = 116345, budget = 1158879
C = 150, clicks = 161, impression = 115939, budget = 1139548
C = 160, clicks = 161, impression = 115477, budget = 1132611
C = 170, clicks = 162, impression = 114821, budget = 1140439
C = 180, clicks = 162, impression = 114363, budget = 1142767
C = 190, clicks = 162, impression = 113751, budget = 1159370
C = 200, clicks = 162, impression = 113300, budget = 1164092
C = 210, clicks = 162, impression = 112677, budget = 1183352

    current_bid = 1.5*c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3)))   
Maxium is 156, when c = 54, budget = 41731, impression = 145685

    current_bid = 1.3*c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3)))   
C = 100, clicks = 161, impression = 129256, budget = 568314
C = 110, clicks = 162, impression = 129371, budget = 480364
C = 120, clicks = 162, impression = 129179, budget = 417309
C = 130, clicks = 164, impression = 128918, budget = 378104
C = 140, clicks = 165, impression = 128593, budget = 349523
C = 150, clicks = 165, impression = 127846, budget = 341791
C = 160, clicks = 165, impression = 127305, budget = 336111
C = 170, clicks = 165, impression = 126622, budget = 344070

    current_bid = 1.4*c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3))) 
C = 60, clicks = 154, impression = 136327, budget = 581753
C = 70, clicks = 157, impression = 139571, budget = 231136
C = 80, clicks = 160, impression = 140676, budget = 9281
C = 90, clicks = 154, impression = 137891, budget = 0
C = 100, clicks = 152, impression = 135698, budget = 0

    current_bid = 1.35*c*((((ctr+(c**2 * lamda**2 + ctr**2)**0.5)/(c*lamda))**(1/3))       #non_linear 1
                     -(((c*lamda)/(lamda + (c**2 * lamda**2 + ctr**2)**0.5))**(1/3))) 
C = 70, clicks = 157, impression = 132760, budget = 662579
C = 80, clicks = 158, impression = 134795, budget = 416464
C = 90, clicks = 161, impression = 135381, budget = 255209
C = 100, clicks = 163, impression = 135704, budget = 126594
C = 110, clicks = 164, impression = 135713, budget = 42317
C = 120, clicks = 165, impression = 135255, budget = 0
C = 130, clicks = 164, impression = 134044, budget = 0

Maxium is 165, when c = 113, budget = 18224, impression = 135703

      current_bid = 1.2 * ((c/lamda) * ctr + c**2) **0.5 -c   #non_linear 2
C = 10, clicks = 78, impression = 52307, budget = 5306639
C = 20, clicks = 97, impression = 78403, budget = 4401696
C = 30, clicks = 112, impression = 95776, budget = 3523046
C = 40, clicks = 122, impression = 109175, budget = 2750796
C = 50, clicks = 137, impression = 121252, budget = 1985325
C = 60, clicks = 145, impression = 131841, budget = 1319284
C = 70, clicks = 152, impression = 141589, budget = 701860
C = 80, clicks = 158, impression = 150805, budget = 82896
C = 90, clicks = 142, impression = 147348, budget = 0
Maxium is 158, when c = 80, budget = 82896, impression = 150805

C = 78, clicks = 157, impression = 148970, budget = 207037
C = 79, clicks = 157, impression = 149874, budget = 145559
C = 80, clicks = 158, impression = 150805, budget = 82896
C = 81, clicks = 158, impression = 151610, budget = 27793

      current_bid = 1.1 * ((c/lamda) * ctr + c**2) **0.5 -c   #non_linear 2
C = 140, clicks = 160, impression = 144586, budget = 366875
C = 150, clicks = 163, impression = 148755, budget = 89313
C = 160, clicks = 157, impression = 148487, budget = 0

C = 150, clicks = 163, impression = 148755, budget = 89313
C = 151, clicks = 164, impression = 149395, budget = 47811
C = 152, clicks = 164, impression = 149809, budget = 20593
C = 153, clicks = 164, impression = 150071, budget = 0
C = 154, clicks = 162, impression = 149873, budget = 0

      current_bid =  ((c/lamda) * ctr + c**2) **0.5 - 0.8*c   #non_linear 2
C = 50, clicks = 115, impression = 101260, budget = 3247384
C = 60, clicks = 121, impression = 110064, budget = 2744181
C = 70, clicks = 130, impression = 118777, budget = 2216692
C = 80, clicks = 137, impression = 127800, budget = 1648750
C = 90, clicks = 144, impression = 136764, budget = 1100869
C = 100, clicks = 153, impression = 144014, budget = 645192
C = 110, clicks = 155, impression = 151834, budget = 134498
C = 120, clicks = 142, impression = 150469, budget = 0

      current_bid =  ((c/lamda) * ctr + c**2) **0.5 - 0.9*c   #non_linear 2
C = 110, clicks = 140, impression = 120532, budget = 1938723
C = 120, clicks = 142, impression = 124308, budget = 1705293
C = 130, clicks = 147, impression = 128324, budget = 1458374
C = 140, clicks = 150, impression = 132710, budget = 1185749
C = 150, clicks = 152, impression = 136463, budget = 951505
C = 160, clicks = 156, impression = 140394, budget = 706907
C = 170, clicks = 160, impression = 144115, budget = 465969
C = 180, clicks = 161, impression = 147845, budget = 222083
C = 190, clicks = 160, impression = 150991, budget = 0

      current_bid =  ((c/lamda) * ctr + c**2) **0.5 - c + 20  #non_linear 2
C = 110, clicks = 153, impression = 145708, budget = 512213
C = 120, clicks = 155, impression = 147351, budget = 382719
C = 130, clicks = 156, impression = 148554, budget = 285534
C = 140, clicks = 157, impression = 149928, budget = 173771
C = 150, clicks = 159, impression = 151003, budget = 85334
C = 160, clicks = 162, impression = 152031, budget = 1
C = 170, clicks = 159, impression = 151178, budget = 0
C = 180, clicks = 156, impression = 150327, budget = 0
'''